In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import logging
# Import thư viện SARIMAX từ statsmodels để dự báo chuỗi thời gian
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Tắt các cảnh báo và log không cần thiết để kết quả gọn gàng
import warnings
warnings.filterwarnings("ignore")

# Sử dụng một backend cụ thể cho matplotlib để tránh lỗi
plt.switch_backend('Agg')

print("Bat dau qua trinh xu ly...")

# --- 1. Tải và Chuẩn bị Dữ liệu ---
try:
    orders_df = pd.read_csv("blinkit_orders_cleaned.csv")
    customers_df = pd.read_csv("blinkit_customers_cleaned.csv")
    area_df = pd.read_csv("India_pincode.csv")
except FileNotFoundError as e:
    print(f"Loi: Khong tim thay file. Vui long kiem tra lai ten file. {e}")
    # Trong môi trường thực tế, bạn có thể muốn dừng ở đây
except Exception as e:
    print(f"Loi khi doc file: {e}")

# --- 2. Gộp Dữ liệu ---
# Chọn các cột cần thiết
orders_data = orders_df[['order_id', 'customer_id', 'order_date']]
customers_data = customers_df[['customer_id', 'area']]
area_data = area_df[['Area', 'City']]

Bat dau qua trinh xu ly...


In [3]:
# Xóa các mã pin trùng lặp để tránh gộp lỗi
area_data = area_data.drop_duplicates(subset=['Area'])

try:
    # Gộp đơn hàng với khách hàng
    orders_with_area = pd.merge(orders_data, customers_data, on='customer_id', how='left')

    # Gộp với dữ liệu mã pin để lấy Thành phố
    merged_df = pd.merge(orders_with_area, area_data, left_on='area', right_on='Area', how='left')

except Exception as e:
    print(f"Loi trong qua trinh gop du lieu: {e}")

# Bỏ qua các đơn hàng không xác định được thành phố
merged_df = merged_df.dropna(subset=['City'])
print("Da gop du lieu thanh cong.")

Da gop du lieu thanh cong.


In [5]:
merged_df.head()

,order_id,customer_id,order_date,area,Area,City
0,1961864118,30065862,2024-07-17 08:34:01,Allahabad,Allahabad,Patan
1,1549769649,9573071,2024-05-28 13:14:29,Thrissur,Thrissur,Trichur
2,9185164487,45477575,2024-09-23 13:07:12,Vellore,Vellore,Pudukkottai
3,9644738826,88067569,2023-11-24 16:16:56,Gaya,Gaya,Gaya
4,5427684290,83298567,2023-11-20 05:00:39,Asansol,Asansol,Santhal Parganas


In [6]:
# --- 3. Tổng hợp Dữ liệu theo Thời gian ---
# Chuyển đổi cột ngày thành đối tượng datetime
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'])
# Tạo một cột mới là 'Tháng' (ví dụ: '2024-05')
merged_df['order_month_period'] = merged_df['order_date'].dt.to_period('M')

# Đếm số đơn hàng theo Thành phố và Tháng
monthly_city_orders = merged_df.groupby(['City', 'order_month_period']).size().reset_index(name='order_count')

# --- 4. Xác định 5 Thành phố hàng đầu ---
total_city_orders = monthly_city_orders.groupby('City')['order_count'].sum().sort_values(ascending=False)
top_cities = total_city_orders.head(5).index
print(f"5 thanh pho co don hang cao nhat: {list(top_cities)}")

# Lấy khoảng thời gian đầy đủ của dữ liệu (từ tháng đầu tiên đến tháng cuối cùng)
try:
    min_month = monthly_city_orders['order_month_period'].min()
    max_month = monthly_city_orders['order_month_period'].max()
    all_months_dt_range = pd.date_range(start=min_month.to_timestamp(), end=max_month.to_timestamp(), freq='MS')
except Exception as e:
    print(f"Loi khi xac dinh pham vi thoi gian: {e}")

all_forecasts_dfs = []
forecast_horizon = 6 # Dự báo 6 tháng tới

print(f"Dang xay dung mo hinh du bao cho {forecast_horizon} thang toi...")



5 thanh pho co don hang cao nhat: ['Nandyal', 'Tambaram', 'Indore Moffusil', 'Cachar', 'Bhojpur']
Dang xay dung mo hinh du bao cho 6 thang toi...


In [10]:
# --- 5. Xây dựng Mô hình và Dự báo ---
for city in top_cities:
    # Lọc dữ liệu cho thành phố hiện tại
    city_data = monthly_city_orders[monthly_city_orders['City'] == city]

    # Chuẩn bị dữ liệu (đổi tên cột cho dễ hiểu)
    city_data_ts = city_data.rename(columns={'order_month_period': 'ds', 'order_count': 'y'})
    city_data_ts['ds'] = city_data_ts['ds'].dt.to_timestamp()

    # Tạo một khung dữ liệu chứa TẤT CẢ các tháng trong lịch sử
    all_months_df = pd.DataFrame({'ds': all_months_dt_range})
    # Gộp với dữ liệu thực tế, những tháng không có đơn hàng sẽ được điền là 0
    city_data_full = pd.merge(all_months_df, city_data_ts[['ds', 'y']], on='ds', how='left').fillna(0)
    city_data_full = city_data_full.set_index('ds')

    # Tạo 6 mốc thời gian trong tương lai để dự báo
    future_dates = pd.date_range(start=city_data_full.index[-1] + pd.DateOffset(months=1), periods=forecast_horizon, freq='MS')

    # Sao chép dữ liệu lịch sử để vẽ biểu đồ
    historical_data_for_plot = city_data_full.copy()

    try:
        # Khởi tạo mô hình SARIMAX
        # (p,d,q)=(1,1,1) là các tham số cho phần không mùa vụ (AR, I, MA)
        # (P,D,Q,m)=(1,1,0,12) là các tham số cho phần mùa vụ (chu kỳ 12 tháng)
        model = SARIMAX(city_data_full['y'],
                        order=(1, 1, 1),
                        seasonal_order=(1, 1, 0, 12),
                        enforce_stationarity=False,
                        enforce_invertibility=False)

        # Huấn luyện (fit) mô hình
        results = model.fit(disp=False)

        # Lấy kết quả dự báo cho 6 bước (tháng) tới
        forecast_obj = results.get_forecast(steps=forecast_horizon)
        predicted_mean = forecast_obj.predicted_mean
        conf_int = forecast_obj.conf_int(alpha=0.05) # Khoảng tin cậy 95%

        # Tạo DataFrame chứa kết quả dự báo
        forecast_df = pd.DataFrame({
            'Thang_Du_Bao': future_dates,
            'So_Don_Du_Kien': predicted_mean.values,
            'Du_Kien_Thap': conf_int.iloc[:, 0].values,
            'Du_Kien_Cao': conf_int.iloc[:, 1].values
        })

        # Dữ liệu để vẽ biểu đồ
        forecast_for_plot = predicted_mean
        conf_int_for_plot = conf_int

    except Exception as e:
        print(f"Loi khi chay SARIMAX cho {city}: {e}. Bo qua thanh pho nay.")
        continue # Bỏ qua thành phố này nếu mô hình bị lỗi

    # Làm sạch dữ liệu dự báo (làm tròn, đảm bảo không có số âm)
    forecast_df['So_Don_Du_Kien'] = forecast_df['So_Don_Du_Kien'].round().astype(int).apply(lambda x: max(0, x))
    if 'Du_Kien_Thap' in forecast_df.columns and pd.api.types.is_numeric_dtype(forecast_df['Du_Kien_Thap']):
        forecast_df['Du_Kien_Thap'] = forecast_df['Du_Kien_Thap'].round().astype(int).apply(lambda x: max(0, x))
    if 'Du_Kien_Cao' in forecast_df.columns and pd.api.types.is_numeric_dtype(forecast_df['Du_Kien_Cao']):
        forecast_df['Du_Kien_Cao'] = forecast_df['Du_Kien_Cao'].round().astype(int).apply(lambda x: max(0, x))

    forecast_df['City'] = city
    all_forecasts_dfs.append(forecast_df)

    # --- 6. Vẽ và Lưu Biểu đồ ---
    try:
        plt.figure(figsize=(10, 6))
        # Vẽ 24 tháng lịch sử gần nhất cho rõ ràng
        plt.plot(historical_data_for_plot.index[-24:], historical_data_for_plot['y'][-24:], label='Lich su (24 thang)')
        # Vẽ đường dự báo
        plt.plot(forecast_for_plot.index, forecast_for_plot.values, label='Du bao', color='red', linestyle='--')

        # Vẽ vùng khoảng tin cậy
        if conf_int_for_plot is not None:
            plt.fill_between(conf_int_for_plot.index,
                             conf_int_for_plot.iloc[:, 0],
                             conf_int_for_plot.iloc[:, 1],
                             color='pink', alpha=0.5, label='Khoang tin cay 95%')

        # Tạo tên tệp an toàn
        city_filename = re.sub(r'[^a-zA-Z0-9_]', '_', city)
        plot_filename = f'forecast_plot_{city_filename}.png'

        plt.title(f'Du bao don hang cho {city}')
        plt.xlabel('Ngay')
        plt.ylabel('So luong don hang')
        plt.legend()
        plt.grid(True)
        # Lưu biểu đồ
        plt.savefig(plot_filename)
        plt.close() # Đóng biểu đồ để tiết kiệm bộ nhớ
        print(f"Da luu bieu do cho {city} tai {plot_filename}")
    except Exception as e:
        print(f"Loi khi ve bieu do cho {city}: {e}")
        plt.close()

Da luu bieu do cho Nandyal tai forecast_plot_Nandyal.png
Da luu bieu do cho Tambaram tai forecast_plot_Tambaram.png
Da luu bieu do cho Indore Moffusil tai forecast_plot_Indore_Moffusil.png
Da luu bieu do cho Cachar tai forecast_plot_Cachar.png
Da luu bieu do cho Bhojpur tai forecast_plot_Bhojpur.png


In [11]:
# --- 7. In Kết quả ---
print("\n--- KET QUA DU BAO (6 THANG TOI) ---")
if not all_forecasts_dfs:
    print("Khong co ket qua du bao nao duoc tao.")
else:
    # Gộp tất cả các dự báo vào một bảng lớn
    final_forecast_table = pd.concat(all_forecasts_dfs).reset_index(drop=True)
    # Định dạng lại ngày cho dễ đọc
    final_forecast_table['Thang_Du_Bao'] = final_forecast_table['Thang_Du_Bao'].dt.strftime('%Y-%m')

    # Sắp xếp lại cột
    final_forecast_table = final_forecast_table[['City', 'Thang_Du_Bao', 'So_Don_Du_Kien', 'Du_Kien_Thap', 'Du_Kien_Cao']]

    # In kết quả cho từng thành phố
    for city in top_cities:
        if city in final_forecast_table['City'].values:
            print(f"\n=== {city} ===")
            print(final_forecast_table[final_forecast_table['City'] == city].to_string(index=False))

print("\nHoan thanh qua trinh phan tich va du bao.")


--- KET QUA DU BAO (6 THANG TOI) ---

=== Nandyal ===
   City Thang_Du_Bao  So_Don_Du_Kien  Du_Kien_Thap  Du_Kien_Cao
Nandyal      2024-12               3             0            8
Nandyal      2025-01               1             0            6
Nandyal      2025-02               3             0           10
Nandyal      2025-03               5             0           12
Nandyal      2025-04               3             0           10
Nandyal      2025-05               0             0            8
Nandyal      2024-12               3             0            8
Nandyal      2025-01               1             0            6
Nandyal      2025-02               3             0           10
Nandyal      2025-03               5             0           12
Nandyal      2025-04               3             0           10
Nandyal      2025-05               0             0            8

=== Tambaram ===
    City Thang_Du_Bao  So_Don_Du_Kien  Du_Kien_Thap  Du_Kien_Cao
Tambaram      2024-12         

In [14]:
final_forecast_table.to_csv("forecast_sum_order_per_city.csv")